### SQL Part 1 - Basic Queries

In [ ]:
SELECT name
FROM dogs
WHERE ownerid = 3;

In [ ]:
SELECT name, age
FROM dogs
ORDER BY age DESC, name
LIMIT 5;

In [ ]:
SELECT breed, COUNT(*) as total
FROM dogs
GROUP BY breed
HAVING COUNT(*) > 1;

### SQL Part 2 - Joins & Subqueries

In [ ]:
SELECT d.name
FROM dogs as d INNER JOIN users as u
ON d.ownerid = u.userid
WHERE u.name = "Josh Hug";

In [ ]:
SELECT d.name
FROM dogs as d, users as u
WHERE d.ownerid = u.userid AND u.name = "Josh Hug";

In [ ]:
WITH ownerDogsCount AS (
    SELECT u.name, COUNT(*) as numDogs -- Does u.userid need to be added here?
    FROM dogs as d INNER JOIN users as u
    ON d.ownerid = u.userid
    GROUP BY u.name, u.userid -- Users may have the same name
)

SELECT u.name, numDogs
FROM ownerDogsCount
ORDER BY numDogs DESC
LIMIT 1;

SELECT users.name, COUNT(*)
FROM users INNER JOIN dogs on users.userid = dogs.ownerid
GROUP BY users.userid, users.name
ORDER BY COUNT(*) DESC
LIMIT 1;

In [ ]:
WITH ownerDogsCount AS (
    SELECT u.name, COUNT(*) as numDogs
    FROM dogs as d INNER JOIN users as u
    ON d.ownerid = u.userid
    GROUP BY u.name, u.userid
)

SELECT u.name, numDogs
FROM ownerDogsCount
WHERE numDOGS = (
    SELECT MAX(numDogs)
    FROM ownerDogsCount
);

SELECT users.name, COUNT(*)
FROM users INNER JOIN dogs ON users.userid = dogs.ownerid
GROUP BY users.userid, users.name
HAVING COUNT(*) >= all( -- Can you use MAX instead of all?
    SELECT COUNT(*)
    FROM dogs
    GROUP BY ownerid
);


### Disks and Files
- READ: data from disk -> RAM
- WRITE: data from RAM -> disk
- Basic unit of data for a disk is a PAGE
- HEAP file: page directory better than linked list for insertion; time complexity for searching is linear (N I/O) as unordered
- SORTED file: better at searching (logN I/O) where N is the # pages (uses binary search); insertion in average case is (logN + N I/O) as you first have to search for page and then on average N/2 pages will need to be pushed back; as these need both read and write I/O, there will be on average N I/O operations

1. I/O Cost: 4 * 1 (read header) + 1 (read data) + 1 (write data) + 1 (write last header) = 7 I/Os

2. 5 (Record Header) ~~+ 1 (VARCHAR)~~ + 1 (BOOLEAN) + 8 (DATE) = ~~15~~ 14 bytes
- VARCHAR (name/state) can be NULL

3. ~~1024//14 = 73~~
In order to maximize the number of records that can be stored, we will consider the case when all records are of the minimum size. In the previous question, the minimum size of the record was calculated to be 14 bytes. Additionally, we need to account for the slot directory entires and the rest of the page footer, containing a free space pointer and a slot count. The slot count is 4B, the free space pointer is 4B, and the slot directory entries are 8B per record.

Thus, the max number of records that can be stored is floor (1024 - 8 / 14 + 8)

4. 5 (Record Header) + 12 (VARCHAR(12)) + 1 (BOOLEAN) + 8 (DATE) + 2 (VARCHAR(2)) = 28 bytes

5. 4 (slot count) + 4 (free space pointer) + 4 * 8 (entry) = 40B
- entry is record pointer (4B) + record length (4B)

6. 1 (read header) + 5 (read pages with free space) = 6 I/Os
- Full page portion is not read

7. False - in a page directory implementation metadata is stored in the header page with pointers to associated data pages and the amount of free space left within that data page